In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" > /etc/apt/sources.list.d/tensorflow-serving.list
    !curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
    !apt update && apt-get install -y tensorflow-model-server
    !pip install -q -U tensorflow-serving-api
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deploy"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 텐서플로우 서빙 사용하기

### SavedModel 사용하기

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

11493376/11490434 [==============================] - 1s 0us/step


In [3]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 53s 968us/sample - loss: 0.7031 - accuracy: 0.8237 - val_loss: 0.3718 - val_accuracy: 0.9032
Epoch 2/10
55000/55000 [==============================] - 46s 845us/sample - loss: 0.3532 - accuracy: 0.9012 - val_loss: 0.3000 - val_accuracy: 0.9150
Epoch 3/10
55000/55000 [==============================] - 31s 562us/sample - loss: 0.3031 - accuracy: 0.9145 - val_loss: 0.2667 - val_accuracy: 0.9270
Epoch 4/10
55000/55000 [==============================] - 30s 549us/sample - loss: 0.2732 - accuracy: 0.9236 - val_loss: 0.2442 - val_accuracy: 0.9308
Epoch 5/10
55000/55000 [==============================] - 32s 581us/sample - loss: 0.2504 - accuracy: 0.9297 - val_loss: 0.2256 - val_accuracy: 0.9388
Epoch 6/10
55000/55000 [==============================] - 28s 507us/sample - loss: 0.2315 - accuracy: 0.9350 - val_loss: 0.2106 - val_accuracy: 0.9416
Epoch 7/10
55000/55000 [=====================

In [4]:
np.round(model.predict(X_new), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [5]:
model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
model_path

'my_mnist_model/0001'

In [6]:
!rm -rf {model_name}

In [7]:
#모델의 게산 그래프와 가중치를 저장

tf.saved_model.save(model, model_path)

#또는 모델의 save()메서드 이용 가능
#파일 확장자가 .h5가 아니면 HDF5포맷 대신에 SavedModel 포맷으로 모델을 저장

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


In [8]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))
        
#saved_mode.pd : 계산 그래프를 정의
#variables : 변수값 들
#assets : 부가적인 데이터

my_mnist_model/
    0001/
        saved_model.pb
        assets/
        variables/
            variables.index
            variables.data-00000-of-00002
            variables.data-00001-of-00002


In [11]:
#모델 불러와서 예측하기

saved_model = tf.saved_model.load(model_path)
y_pred = saved_model(tf.constant(X_new, dtype=tf.float32))

np.round(y_pred, 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [12]:
#SavedModel을 케라스 모델로 로드하기

model = keras.models.load_model(model_path)
y_pred = model.predict(X_new)

np.round(y_pred, 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

In [13]:
!saved_model_cli show --dir {model_path}

2020-07-28 05:34:03.785299: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-07-28 05:34:03.787950: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
The given SavedModel contains the following tag-sets:
serve


In [ ]:
!saved_model_cli show --dir {model_path} --tag_set serve

In [ ]:
!saved_model_cli show --dir {model_path} --tag_set serve \
                      --signature_def serving_default

In [14]:
!saved_model_cli show --dir {model_path} --all

2020-07-28 05:34:17.446193: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-07-28 05:34:17.448083: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following outp

In [15]:
#numpy 배열을 npy포맷으로 저장

np.save("my_mnist_tests.npy", X_new)

In [16]:
input_name = model.input_names[0]
input_name

'flatten_input'

In [17]:
!saved_model_cli run --dir {model_path} --tag_set serve \
                     --signature_def serving_default    \
                     --inputs {input_name}=my_mnist_tests.npy

2020-07-28 05:35:28.100302: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-07-28 05:35:28.102259: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
2020-07-28 05:35:28.994528: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-28 05:35:29.059811: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2020-07-28 05:35:29.060804: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 950M computeCapability: 5.0
coreClock: 1.124GHz coreCount: 5 deviceMemorySize: 2.00GiB deviceMemoryBandwidth: 29.83GiB/s
2020-07-28 05:35:29.060982: I tensorflow/stream_executor/pl

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py", line 1367, in _do_call
    return fn(*args)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py", line 1352, in _run_fn
    target_list, run_metadata)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py", line 1445, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(3, 784), b.shape=(784, 100), m=3, n=100, k=784
	 [[{{node sequential/dense/MatMul}}]]
	 [[StatefulPartitionedCall/_49]]
  (1) Internal: Blas GEMM launch failed : a.shape=(3, 784), b.shape=(784, 100), m=3, n=100, k=784
	 [[{{node sequential/dense/MatMul}}]]
0 successful operations.
0 derived errors ignored.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  Fi

# 텐서플로우 서빙 이용하기

도커로 실행해야 함

위에서 저장한 파일을 /models/my_mnist_model로 복사하였음

sudo docker cp tensorflow:/tf/python-projects/deeplearning/deeplearning/"핸즈온 머신러닝 2e"/my_mnist_model/ .

cp -r ../my_mnist_model/ ./my_mnist_model

sudo docker run -it --rm -p 8500:8500 -p 8501:8501 -v "/mnt/c/Users/netsc/models/my_mnist_model:/models/my_mnist_model" -e MODEL_NAME=my_mnist_model tensorflow/serving


8500번 포트는 gRPC API

8501번 포트는 REST API

In [18]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [19]:
repr(input_data_json)[:1500] + "..."

'\'{"signature_name": "serving_default", "instances": [[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0

In [ ]:
#현재 도커 컨테이너 안에서 주피터 노트북을 실행 중이므로 다른 도커 컨테이너에 대해서 통신하려면
#docker-compose로 네트워크 브리지를 만들어야 한다.

#그냥 실행하고 싶으면 코드를 파이썬 파일로 저장하고 호스트 컴퓨터에서 실행 할 것!

import requests

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [ ]:
response.keys()

In [ ]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

In [ ]:
#파이썬 파일로 실행한 코드

import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import requests

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_new = X_test[:3]

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

print(response.keys())

y_proba = np.array(response["predictions"])
print(y_proba.round(2))

실행 결과

~~~

dict_keys(['predictions'])

[[0.   0.   0.   0.   0.   0.   0.   1.   0.   0.  ]  
 [0.   0.   0.99 0.01 0.   0.   0.   0.   0.   0.  ]  
 [0.   0.97 0.01 0.   0.   0.   0.   0.01 0.   0.  ]]

~~~